In [1]:
import enum
import numpy as np
import PIL
import os
import functools
from PIL import Image
import matplotlib.pyplot as plt
import scipy.stats

## Loading data

In [2]:
class NoiseType(enum.Enum):
    INDEPENDENT = "independent"
    GAUSSIAN = "gaussian"


class ImageType(enum.Enum):
    GROUND_TRUTH = "ground_truth"
    OBSERVATION = "observation"

In [3]:
TARGET_SIZE = 800

In [4]:
def show_image(image):
    plt.figure(figsize=(12, 8))
    plt.tick_params(labelbottom=False, labelleft=False)    
    plt.imshow(image, cmap='gray')


def load_ground_truth_images():
    images = []
    for image_id in range(20):
        image = Image.open(f"grayscale_images/raw/{image_id}.jpeg")
        if image.width > image.height:
            target_width = TARGET_SIZE
            target_height = int(image.height * (TARGET_SIZE / image.width))
        else:
            target_width = int(image.width * (TARGET_SIZE / image.height))
            target_height = TARGET_SIZE
        image = image.resize(size=(target_width, target_height)).convert(mode='L')
        images.append(image)
    return images


def save_image(image_data, image_id, image_type, noise_type, noise_level, max_noise_distance):
    noise_level_repr = int(round(100 * noise_level))
    noise_repr = f"noise{noise_level_repr}_{noise_type.value}_distance{max_noise_distance}"
    path = os.path.join(
        "grayscale_images",
        f"size{max(image_data.width, image_data.height)}_{noise_repr}",
        f"image_{image_id}_{image_type.value}.png",
    )
    os.makedirs(os.path.dirname(path), exist_ok=True)
    image_data.save(path, format="PNG")

In [5]:
ground_truth_images = load_ground_truth_images()
len(ground_truth_images)

20

## Generating noise

In [6]:
def generate_independent_noise(image, noise_level, max_noise_distance):
    array_of_image = np.array(image).astype(np.int32)
    noisy_indices = np.random.choice(
        a=[False, True],
        size=array_of_image.shape,
        p=[1.0 - noise_level, noise_level]
    )
    noisy_pixels = array_of_image[noisy_indices].copy()
    noise = np.random.randint(
        low=-max_noise_distance, high=(max_noise_distance + 1), size=noisy_pixels.size
    )
    noisy_pixels += noise
    noisy_pixels[noisy_pixels < 0] = 0
    noisy_pixels[noisy_pixels > 255] = 255
    array_of_image[noisy_indices] = noisy_pixels
    return PIL.Image.fromarray(array_of_image.astype(np.uint8))


@functools.lru_cache(maxsize=None)
def gaussian_distribution(point):
    distribution = scipy.stats.multivariate_normal(mean=(0, 0), cov=5)
    return distribution.pdf(point)


def generate_gaussian_noise(image, noise_level, max_noise_distance):
    array_of_image = np.array(image).astype(np.int32)
    noise_center_strength = gaussian_distribution((0, 0))
    for _ in range(int(noise_level * array_of_image.size)):
        noise_center = (
            np.random.randint(0, image.height),
            np.random.randint(0, image.width),
        )
        noise_center_distance = np.random.randint(
            low=-max_noise_distance, high=max_noise_distance + 1
        )
        for offset_x in range(-7, 8):
            for offset_y in range(-5 + abs(offset_x), 5 - abs(offset_x) + 1):
                noise_position = (
                    noise_center[0] - offset_x, noise_center[1] - offset_y
                )
                if noise_position[0] < 0 or noise_position[0] >= array_of_image.shape[0]:
                    continue
                if noise_position[1] < 0 or noise_position[1] >= array_of_image.shape[1]:
                    continue
                noise_absolute_strength = gaussian_distribution((offset_x, offset_y))
                noise_relative_strength = noise_absolute_strength / noise_center_strength
                noise_distance = int(noise_relative_strength * noise_center_distance)
                array_of_image[noise_position] += noise_distance
    array_of_image[array_of_image < 0] = 0
    array_of_image[array_of_image > 255] = 255
    return PIL.Image.fromarray(array_of_image.astype(np.uint8))


def generate_noise(image, noise_type, noise_level, max_noise_distance):
    if noise_type == NoiseType.INDEPENDENT:
        return generate_independent_noise(image, noise_level, max_noise_distance)
    elif noise_type == NoiseType.GAUSSIAN:
        return generate_gaussian_noise(image, noise_level, max_noise_distance)
    else:
        raise ValueError(f"Invalid noise type: {noise_type}")

## Running developed generator

In [7]:
def generate_for_params(images, noise_type, noise_level, max_noise_distance):
    for image_id, image in enumerate(images):
        save_image(
            image, image_id, ImageType.GROUND_TRUTH,
            noise_type, noise_level, max_noise_distance
        )  
        observation = generate_noise(image, noise_type, noise_level, max_noise_distance)
        save_image(
            observation, image_id, ImageType.OBSERVATION,
            noise_type, noise_level, max_noise_distance
        )  


def run_generator(images):
    for noise_type in [NoiseType.GAUSSIAN, NoiseType.INDEPENDENT]:
        noise_levels = (
            [0.1, 0.15, 0.2, 0.25] if noise_type == NoiseType.INDEPENDENT
            else [0.01, 0.02, 0.03, 0.05]
        )
        for noise_level in noise_levels:           
            for max_noise_distance in [50, 100, 150, 200, 255]:
                generate_for_params(images, noise_type, noise_level, max_noise_distance)        

In [8]:
run_generator(ground_truth_images)